In [ ]:
#Cellule 1 : Imports

#Import de PyTorch pour manipuler les tenseurs et les modèles
import torch

#Import de modèles CNN pré-entraînés comme ResNet, Inception, etc.
import torchvision.models as models

#Import de fonctions de transformation d'image (resize, normalisation, etc.)
import torchvision.transforms as transforms

#PIL (Python Imaging Library) pour ouvrir les fichiers image
from PIL import Image

#Matplotlib pour afficher les images dans le notebook
import matplotlib.pyplot as plt

#Import nécessaire pour manipuler les chemins de fichiers
from pathlib import Path 

from tqdm import tqdm
import numpy as np

In [214]:
#Définition des dossiers d'entrée et de sortie pour les images et les features extraits

#Dossier contenant les images brutes (Flickr8k dataset)
image_folder = Path("../data/raw/Flicker8k_Dataset")

#Dossier où seront enregistrées les features globales extraites par InceptionV3
global_out = Path("../data/processed/features_resnet_global")

#Dossier où seront enregistrées les features spatiales (avant pooling) extraites par Inception V3
spatial_out = Path("../data/processed/features_resnet_spatial")

#Création des dossiers de sortie s'ils n'existent pas déjà
global_out.mkdir(parents=True, exist_ok=True)
spatial_out.mkdir(parents=True, exist_ok=True)

In [215]:
#Choix du device : GPU (cuda) si disponible, sinon CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [216]:
#Charger InceptionV3 complet pré-entraîné (avec aux_logits=True requis pour ce modèle)
full_inception = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, aux_logits=True)

#Mettre sur device et mode évaluation
full_inception = full_inception.to(device).eval()

#Créer une version tronquée du modèle pour extraire les features spatiales
#Ici on prend tous les modules jusqu'à (et incluant) 'Mixed_7c' qui correspond à l'index 15 dans children()
#Note : 'full_inception.children()' renvoie un itérateur sur les modules du modèle dans l'ordre
inception_spatial = torch.nn.Sequential(*list(full_inception.children())[:15])
inception_spatial = inception_spatial.to(device).eval()

In [217]:
#Transformations adaptées à InceptionV3
transform = transforms.Compose([

    transforms.Resize((299, 299)),       
    #Redimensionne l’image à 299x299 pixels.
    #InceptionV3 a été entraîné avec cette taille comme entrée (au lieu de 224x224 pour ResNet).
    #Si tu mets une taille différente, le modèle ne fonctionnera pas correctement.

    transforms.ToTensor(),               
    #Convertit l’image PIL (format image classique) en tenseur PyTorch.
    #Résultat : image sous forme de tenseur [C, H, W] avec valeurs entre 0 et 1 (float32).
    #PyTorch a besoin de cette structure pour les CNN.

    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    #Normalise chaque canal (R, G, B) avec la moyenne et l’écart-type utilisés pour ImageNet.
    #Pourquoi ? Le modèle a été entraîné avec des images normalisées → il s’attend à des entrées avec la même "distribution".
    #Ça stabilise et accélère l’apprentissage et l’inférence.
])

In [218]:
#InceptionV3 a les sous-modules listés dans `full_inception.children()`
#Mais ici on veut récupérer la sortie de Mixed_7c, on peut créer un forward hook dessus

extracted_spatial_features = None

def spatial_hook(module, input, output):
    global extracted_spatial_features
    extracted_spatial_features = output  #shape attendue (1, 2048, 8, 8)

#Attacher hook à la couche Mixed_7c
hook_handle = full_inception.Mixed_7c.register_forward_hook(spatial_hook)


In [219]:
#Sélectionner un sous-échantillon des images (ici les 10 premières .jpg)
image_list = list(image_folder.glob("*.jpg"))[:10]

#Pourquoi ? → pour tester ton pipeline sans traiter tout le dataset
#Utilise une limite faible au début pour gagner du temps


In [220]:
#Prendre un sous-échantillon (par exemple 10 images)
image_list = list(image_folder.glob("*.jpg"))[:10]

In [ ]:
try:
    for img_path in tqdm(image_list, desc="Extraction features InceptionV3"):

        #Charger image
        image = Image.open(img_path).convert("RGB")
        tensor = transform(image).unsqueeze(0).to(device)

        #Reset la variable spatiale
        extracted_spatial_features = None

        with torch.no_grad():
            output = full_inception(tensor)  #output logits (1000,), pas utilisé

            #Extracted_spatial_features doit être rempli par hook sur Mixed_7c
            spatial_feat = extracted_spatial_features.squeeze(0)  # (2048, 8, 8)
            spatial_feat = spatial_feat.permute(1, 2, 0).reshape(-1, 2048)  # (64, 2048)

            #Feature globale par avgpool sur la feature map spatiale
            global_feat = torch.nn.functional.adaptive_avg_pool2d(extracted_spatial_features, (1, 1)).squeeze()  # (2048,)

        #Convertir en numpy
        global_feat_np = global_feat.cpu().numpy()
        spatial_feat_np = spatial_feat.cpu().numpy()

        #Affichage des tailles
        print(f"Image: {img_path.stem}")
        print(f" - Global feature shape: {global_feat.shape}")         #(2048,)
        print(f" - Spatial feature shape: {spatial_feat.shape}")       #(64, 2048)
        print(f" - Exemple valeurs globales (5 premières): {global_feat_np[:5]}")
        print(f" - Exemple valeurs spatiales (patch 0, 5 premières dim): {spatial_feat_np[0, :5]}")

        #Sauvegarder
        image_id = img_path.stem
        np.save(global_out / f"{image_id}.npy", global_feat_np)
        np.save(spatial_out / f"{image_id}.npy", spatial_feat_np)

finally:
    hook_handle.remove()

print("Extraction terminée, features globales et spatiales sauvegardées !")


Extraction features InceptionV3:  40%|████      | 4/10 [00:00<00:00, 17.70it/s]

Image: 2387197355_237f6f41ee
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.55176234 0.45867205 0.33790833 0.3100482  0.03113861]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.         0.         0.         0.14213018 0.        ]
Image: 2609847254_0ec40c1cce
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.6243857  0.81712115 0.6208783  0.20508268 0.3104276 ]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.01160201 0.         0.3181781  0.         0.        ]
Image: 2046222127_a6f300e202
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.10810361 0.24628781 0.37049508 0.36082956 0.05471717]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.         0.         0.         0.23917572 0.10

Extraction features InceptionV3: 100%|██████████| 10/10 [00:00<00:00, 18.68it/s]

Image: 3421131122_2e4bde661e
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.13459723 0.3667224  0.28416947 0.1908385  0.06326994]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.         0.06548527 0.6173545  0.08319648 0.        ]
Image: 3229730008_63f8ca2de2
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.30081764 0.40682164 0.12245138 0.5416552  0.6263184 ]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.939492 0.       0.       0.       0.      ]
Image: 3220009216_10f088185e
 - Global feature shape: torch.Size([2048])
 - Spatial feature shape: torch.Size([64, 2048])
 - Exemple valeurs globales (5 premières): [0.44251522 0.43607864 0.10273524 0.40928122 0.97594726]
 - Exemple valeurs spatiales (patch 0, 5 premières dim): [0.12612928 0.         0.         0.         0.16534363]
Im